In [1]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

mndata = MNIST('./')
images_train, labels_train = mndata.load_training()
images_test, labels_test = mndata.load_testing()

images_test=np.array(images_test)
labels_test = np.array(labels_test)

images_train1 = np.array(images_train)
labels_train1 = np.array(labels_train)
solvers=["lbfgs"]


/usr/lib/python3/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def classification_report_to_dataframe2(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-2:-1]:
        row = {}
        p = re.compile("avg / total")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [3]:
excel_file_name='Logistic_examples_' + str(datetime.now().strftime('%Y%m%d%H%m%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df=pd.DataFrame(columns=['Examples','Accuracy','Normalized Accuracy',
                         'Time_taken_train','Time_taken_test','Negative Log loss',
                         'Macro_auc','Micro_auc'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [4]:
scaler=StandardScaler()

In [8]:
print(labels_train1)

[5 0 4 ... 5 6 8]


In [6]:
ppca=[1,2,5,10,20,30,50,75,100,150,200,250,300,325,350,400,450,500,550,600,700,784]
ppca=[1]
set=[100,200,250,300,500,750,1000,2000,2500,5000,10000,20000,30000,40000,50000,60000]
for t in set:
    images_train=images_train1[0:t]
    labels_train=labels_train1[0:t]
    for p in ppca:

        #scaler.fit(images_train)
        #print('For',p)
        # Apply transform to both the training set and the test set.
        #train_img = scaler.transform(images_train)
        #test_img = scaler.transform(images_test)

        #pca = PCA(n_components=p)
        #pca.fit(train_img)


        #train_img = pca.transform(train_img)
        #test_img = pca.transform(test_img
        
        train_img=images_train
        test_img=images_test

        s=solvers[0]
        classifier=LogisticRegression(solver=s)
        print("training started for ",s)
        start=time()
        classifier.fit(train_img,labels_train)
        train_time=time()-start
        print("trained")
        print("prediction started")
        start=time()
        predictions=classifier.predict(test_img)
        test_time=time()-start
        print("predicted")
        accuracy=accuracy_score(predictions,labels_test,normalize=False)
        normalized_accuracy=accuracy_score(predictions,labels_test,normalize=True)
        log_loss_prediction=classifier.predict_proba(test_img)
        log_loss_val=log_loss(labels_test,log_loss_prediction)
        classification_repo=classification_report(labels_test,predictions)
        print(classification_repo)

        labels_test_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
        predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
        micro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="micro")
        macro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="macro")

        cm = confusion_matrix(labels_test,predictions)
        cm_df = pd.DataFrame(data=cm)

        print(micro_auc,macro_auc)

        df_data=[t,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
        df.loc[len(df)]=df_data
        df_classification_report=classification_report_to_dataframe2(classification_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='logistic_cr_')
        book = load_workbook(excel_file_name)

        print("writing")
        with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
            excel_writer.book = book
            #df_classification_report = classification_report_to_dataframe2(classfication_repo)
            df_classification_report.to_excel(excel_writer, sheet_name='logistic_cr_' +str(t)+'_'+ str(p))
            cm_df.to_excel(excel_writer, sheet_name='logistic_cr_' +str(t)+'_'+ str(p), 
                           startrow=len(df_classification_report)+5)
            excel_writer.save()
            print('w')

print("done for all solvers")
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='logistic_cr_')
    excel_writer.save()
    print('w')


training started for  lbfgs
trained
prediction started
predicted
             precision    recall  f1-score   support

          0       0.82      0.88      0.85       980
          1       0.72      0.95      0.82      1135
          2       0.75      0.57      0.65      1032
          3       0.70      0.78      0.74      1010
          4       0.50      0.82      0.62       982
          5       0.68      0.29      0.40       892
          6       0.80      0.74      0.77       958
          7       0.75      0.79      0.77      1028
          8       0.62      0.50      0.55       974
          9       0.50      0.41      0.45      1009

avg / total       0.68      0.68      0.67     10000

0.8212222222222222 0.8175811934720161
writing
w


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


training started for  lbfgs
trained
prediction started
predicted
             precision    recall  f1-score   support

          0       0.84      0.89      0.87       980
          1       0.83      0.94      0.88      1135
          2       0.74      0.72      0.73      1032
          3       0.72      0.77      0.75      1010
          4       0.67      0.84      0.74       982
          5       0.67      0.34      0.45       892
          6       0.84      0.71      0.77       958
          7       0.76      0.85      0.80      1028
          8       0.65      0.65      0.65       974
          9       0.64      0.60      0.62      1009

avg / total       0.74      0.74      0.73     10000

0.8549999999999999 0.8514454780644016
writing
w


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


training started for  lbfgs
trained
prediction started
predicted
             precision    recall  f1-score   support

          0       0.85      0.93      0.89       980
          1       0.81      0.95      0.87      1135
          2       0.79      0.73      0.76      1032
          3       0.74      0.77      0.75      1010
          4       0.77      0.82      0.80       982
          5       0.73      0.47      0.57       892
          6       0.87      0.86      0.86       958
          7       0.78      0.88      0.82      1028
          8       0.69      0.61      0.65       974
          9       0.66      0.65      0.65      1009

avg / total       0.77      0.77      0.77     10000

0.8734999999999999 0.8707722977865332
writing


...

w
training started for  lbfgs
trained
prediction started
predicted
             precision    recall  f1-score   support

          0       0.95      0.98      0.96       980
          1       0.96      0.98      0.97      1135
          2       0.93      0.88      0.90      1032
          3       0.90      0.91      0.90      1010
          4       0.93      0.93      0.93       982
          5       0.90      0.85      0.87       892
          6       0.94      0.95      0.94       958
          7       0.93      0.92      0.92      1028
          8       0.84      0.88      0.86       974
          9       0.90      0.90      0.90      1009

avg / total       0.92      0.92      0.92     10000

0.9540000000000001 0.9533240897675826
writing


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


w
done for all solvers


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


w
